In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import numpy as np

In [2]:
from label import label

In [3]:
df = label('.//data//binance-BTCUSDT-1m.csv', t=60)

In [4]:
df

,Open,High,Low,Close,Volumn,Timestamp End,Quote_asset_volume,Number_of_trades,Base_asset_volume,Quote_asset_volume,Time_UTC_Start,label
Timestamp,,,,,,,,,,,,
1531911240000,7404.23,7433.58,7388.97,7418.00,1481.578981,1531914839999,1.098155e+07,9393,838.126191,6.212777e+06,2018-07-18 10:54:00,1.0
1531914840000,7417.99,7472.02,7406.23,7454.01,2113.201163,1531918439999,1.573335e+07,11363,1181.473451,8.794959e+06,2018-07-18 11:54:00,1.0
1531918440000,7454.01,7457.25,7386.12,7447.66,2131.028279,1531922039999,1.583119e+07,11790,1061.337854,7.886237e+06,2018-07-18 12:54:00,0.0
1531922040000,7447.66,7531.30,7431.66,7439.88,3355.440987,1531925639999,2.506039e+07,17679,1800.179303,1.344742e+07,2018-07-18 13:54:00,0.0
1531925640000,7438.81,7454.00,7360.40,7415.32,2500.370323,1531929239999,1.853965e+07,13908,1231.974856,9.136573e+06,2018-07-18 14:54:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1651892040000,35968.65,35974.33,35760.00,35889.40,896.214420,1651895639999,3.214347e+07,22739,409.205980,1.467542e+07,2022-05-07 02:54:00,0.0
1651895640000,35889.40,35890.23,35755.85,35819.05,995.509940,1651899239999,3.565444e+07,24661,490.635660,1.757112e+07,2022-05-07 03:54:00,0.0
1651899240000,35819.06,35897.44,35722.24,35858.28,966.414510,1651902839999,3.462155e+07,23920,445.316230,1.595124e+07,2022-05-07 04:54:00,1.0


In [5]:
import gc
def base_feat(df, base_cols=[], 
                  time_delta=1, 
                  window_lookback=[1, 7, 24, 99, 500],
                  agg_func=['min', 'max', 'mean', 'median', 'var']):
    """
    This func use for create formal features base on a list of columns
    Params:
        df (Pandas DF): dataframe input
        base_cols (list): list of columns you want to create feature by this function
        time_delta (int): time between 2 records (minutes)
        window_lookback (list): list of integer time the time_delta. For example time_delta = 15 window_lookback = [1, 2]
                                so the time lookback is 15 minutes and 15*2=30 minutes
        agg_func (list): aggregate func you use for rolling
    """
    
    for lookback in window_lookback:
        for col in base_cols:
            dict_agg_func = {'_'.join([agg, col, str(time_delta * lookback) + 'M']): agg for agg in agg_func}
            df_feat = df[col].rolling(lookback).agg(dict_agg_func)
            df = pd.concat([df, df_feat], axis=1)
            del df_feat
            gc.collect()
    for col in df.columns:
        if '1' in col and 'var' in col:
            del df[col]
    df = df.dropna()
    return df

In [6]:
base_feat(df, base_cols=['Volumn'])

,Open,High,Low,Close,Volumn,Timestamp End,Quote_asset_volume,Number_of_trades,Base_asset_volume,Quote_asset_volume,...,min_Volumn_99M,max_Volumn_99M,mean_Volumn_99M,median_Volumn_99M,var_Volumn_99M,min_Volumn_500M,max_Volumn_500M,mean_Volumn_500M,median_Volumn_500M,var_Volumn_500M
Timestamp,,,,,,,,,,,,,,,,,,,,,
1533707640000,6550.23,6560.61,6507.32,6533.22,2293.459606,1533711239999,1.498485e+07,10413,1246.407609,8.142464e+06,...,333.454234,5887.455864,1630.218181,1402.046829,9.620047e+05,333.454234,7377.008685,1699.779749,1427.037955,9.322916e+05
1533711240000,6533.22,6555.99,6472.01,6481.54,1956.179657,1533714839999,1.274955e+07,9623,958.843732,6.250491e+06,...,333.454234,5887.455864,1642.699428,1409.628096,9.544874e+05,333.454234,7377.008685,1700.728950,1427.037955,9.323270e+05
1533714840000,6481.54,6498.51,6430.00,6490.00,2039.899437,1533718439999,1.320579e+07,11192,1072.608677,6.945294e+06,...,333.454234,5887.455864,1656.360312,1439.248885,9.465979e+05,333.454234,7377.008685,1700.582346,1427.037955,9.322166e+05
1533718440000,6490.00,6527.42,6475.00,6493.32,1677.195316,1533722039999,1.089607e+07,8608,1026.700569,6.670260e+06,...,333.454234,5887.455864,1660.191796,1454.101402,9.452762e+05,333.454234,7377.008685,1699.674681,1427.037955,9.318455e+05
1533722040000,6493.32,6498.95,6430.89,6485.07,1495.032123,1533725639999,9.682562e+06,8738,711.664430,4.609516e+06,...,333.454234,5887.455864,1665.848317,1463.267092,9.401564e+05,333.454234,7377.008685,1695.953863,1427.037955,9.264215e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651892040000,35968.65,35974.33,35760.00,35889.40,896.214420,1651895639999,3.214347e+07,22739,409.205980,1.467542e+07,...,613.883350,20436.382230,2645.951514,1587.074750,8.615277e+06,231.603380,20436.382230,2000.739515,1337.093965,3.342030e+06
1651895640000,35889.40,35890.23,35755.85,35819.05,995.509940,1651899239999,3.565444e+07,24661,490.635660,1.757112e+07,...,613.883350,20436.382230,2644.751830,1587.074750,8.619132e+06,231.603380,20436.382230,2001.871145,1337.093965,3.339107e+06
1651899240000,35819.06,35897.44,35722.24,35858.28,966.414510,1651902839999,3.462155e+07,23920,445.316230,1.595124e+07,...,613.883350,20436.382230,2640.550021,1587.074750,8.631596e+06,231.603380,20436.382230,2002.265652,1337.093965,3.338211e+06
